In [ ]:
pip install scapy pyshark scikit-learn pandas matplotlib tensorflow


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install nest_asyncio


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import asyncio
import pyshark
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

def capture_packets():
    cap = pyshark.LiveCapture(interface='en0')  # Adjust the interface
    for packet in cap.sniff_continuously(packet_count=10):
        print(packet)

# Run the capture function synchronously
capture_packets()


Packet (Length: 79)
Layer ETH
:	Destination: 94:a6:7e:e7:f3:0d
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: 10:bd:3a:77:8a:16
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Type: IPv4 (0x0800)
	Stream index: 0
Layer IP
:	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 65
	Identification: 0x0be4 (3044)
	000. .... = Flags: 0x0
	0... .... = Reserved bit: Not set
	.0.. .... = Don't fragment: Not set
	..0. .... = More fragments: Not set
	...0 0000 0000 0000 = Fragment Offset: 0
	Time to Live: 64
	Protocol: UDP (17)
	Header Checksum: 0xeb3f [val

In [ ]:
import asyncio
import pyshark
import nest_asyncio
import pandas as pd

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Function to extract features from the packets
def extract_features(packet):
    features = {}

    try:
        # Basic Packet Information
        features['timestamp'] = packet.sniff_time
        features['length'] = packet.length

        # Ethernet Layer
        if 'eth' in packet:
            features['src_mac'] = packet.eth.src
            features['dst_mac'] = packet.eth.dst

        # ARP Layer
        if 'arp' in packet:
            features['arp_opcode'] = packet.arp.opcode
            features['arp_src_ip'] = packet.arp.src_proto_ipv4
            features['arp_dst_ip'] = packet.arp.dst_proto_ipv4

        # IPv4 Layer
        if 'ip' in packet:
            features['src_ip'] = packet.ip.src
            features['dst_ip'] = packet.ip.dst
            features['ip_len'] = packet.ip.len
            features['ip_ttl'] = packet.ip.ttl

        # TCP Layer
        if 'tcp' in packet:
            features['src_tcp_port'] = packet.tcp.srcport
            features['dst_tcp_port'] = packet.tcp.dstport
            features['tcp_flags'] = packet.tcp.flags
            features['seq_number'] = packet.tcp.seq
            features['ack_number'] = packet.tcp.ack
            features['tcp_window_size'] = packet.tcp.window_size

        # TLS Layer
        if 'tls' in packet:
            features['tls_record_type'] = packet.tls.record_type
            features['tls_version'] = packet.tls.version

        # UDP Layer
        if 'udp' in packet:
            features['src_udp_port'] = packet.udp.srcport
            features['dst_udp_port'] = packet.udp.dstport

        # Extract any additional protocol-specific features (for future customizations)
        return features

    except AttributeError as e:
        # In case the packet doesn't contain a specific layer
        return None


def capture_packets():
    cap = pyshark.LiveCapture(interface='en0')  # Adjust the interface

    # A list to store extracted features
    feature_list = []

    # Capture a specific number of packets
    for packet in cap.sniff_continuously(packet_count=10):
        features = extract_features(packet)
        if features:
            feature_list.append(features)

    # Convert the feature list into a pandas DataFrame for further analysis
    df = pd.DataFrame(feature_list)

    # Print the DataFrame of features
    print(df)

# Run the capture function synchronously
capture_packets()


                   timestamp length            src_mac            dst_mac  \
0 2025-02-06 20:13:50.277955    172  94:a6:7e:e7:f3:0d  10:bd:3a:77:8a:16   
1 2025-02-06 20:13:50.279424     78  10:bd:3a:77:8a:16  94:a6:7e:e7:f3:0d   
2 2025-02-06 20:13:50.332541     97  10:bd:3a:77:8a:16  94:a6:7e:e7:f3:0d   
3 2025-02-06 20:13:50.332624     97  10:bd:3a:77:8a:16  94:a6:7e:e7:f3:0d   
4 2025-02-06 20:13:50.337621     74  94:a6:7e:e7:f3:0d  10:bd:3a:77:8a:16   
5 2025-02-06 20:13:50.337717     66  10:bd:3a:77:8a:16  94:a6:7e:e7:f3:0d   
6 2025-02-06 20:13:50.361930    283  94:a6:7e:e7:f3:0d  10:bd:3a:77:8a:16   
7 2025-02-06 20:13:50.361935    229  94:a6:7e:e7:f3:0d  10:bd:3a:77:8a:16   
8 2025-02-06 20:13:50.362968    103  10:bd:3a:77:8a:16  94:a6:7e:e7:f3:0d   

          src_ip         dst_ip ip_len ip_ttl src_udp_port dst_udp_port  \
0    192.168.1.1   192.168.1.55    158     64           53        56600   
1   192.168.1.55  20.189.173.26     64     64          NaN          NaN   
2   

In [ ]:
pip install scikit-learn pandas pyshark nest_asyncio


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import asyncio
import pyshark
import nest_asyncio
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Function to convert IP and MAC addresses to numerical values
def ip_to_int(ip):
    return int(ip.split('.')[0]) * 256**3 + int(ip.split('.')[1]) * 256**2 + int(ip.split('.')[2]) * 256 + int(ip.split('.')[3])

def mac_to_int(mac):
    return int(mac.replace(":", ""), 16)

# Function to extract features from packets
def extract_features(packet):
    features = {}

    try:
        # Basic Packet Information
        features['length'] = packet.length

        # Ethernet Layer
        if 'eth' in packet:
            features['src_mac'] = mac_to_int(packet.eth.src)  # Convert MAC address to numeric
            features['dst_mac'] = mac_to_int(packet.eth.dst)  # Convert MAC address to numeric
        else:
            features['src_mac'] = None
            features['dst_mac'] = None

        # ARP Layer
        if 'arp' in packet:
            features['arp_opcode'] = packet.arp.opcode
            features['arp_src_ip'] = ip_to_int(packet.arp.src_proto_ipv4)  # Convert IP to numeric
            features['arp_dst_ip'] = ip_to_int(packet.arp.dst_proto_ipv4)  # Convert IP to numeric
        else:
            features['arp_opcode'] = None
            features['arp_src_ip'] = None
            features['arp_dst_ip'] = None

        # IPv4 Layer
        if 'ip' in packet:
            features['src_ip'] = ip_to_int(packet.ip.src)  # Convert IP to numeric
            features['dst_ip'] = ip_to_int(packet.ip.dst)  # Convert IP to numeric
            features['ip_len'] = packet.ip.len
            features['ip_ttl'] = packet.ip.ttl
        else:
            features['src_ip'] = None
            features['dst_ip'] = None
            features['ip_len'] = None
            features['ip_ttl'] = None

        # TCP Layer
        if 'tcp' in packet:
            features['src_tcp_port'] = int(packet.tcp.srcport)
            features['dst_tcp_port'] = int(packet.tcp.dstport)
            features['tcp_flags'] = packet.tcp.flags
            features['seq_number'] = int(packet.tcp.seq)
            features['ack_number'] = int(packet.tcp.ack)
            features['tcp_window_size'] = int(packet.tcp.window_size)
        else:
            features['src_tcp_port'] = None
            features['dst_tcp_port'] = None
            features['tcp_flags'] = None
            features['seq_number'] = None
            features['ack_number'] = None
            features['tcp_window_size'] = None

        # TLS Layer
        if 'tls' in packet:
            features['tls_record_type'] = int(packet.tls.record_type)
            features['tls_version'] = int(packet.tls.version)
        else:
            features['tls_record_type'] = None
            features['tls_version'] = None

        # UDP Layer
        if 'udp' in packet:
            features['src_udp_port'] = int(packet.udp.srcport)
            features['dst_udp_port'] = int(packet.udp.dstport)
        else:
            features['src_udp_port'] = None
            features['dst_udp_port'] = None

        return features

    except AttributeError:
        # In case the packet doesn't contain a specific layer
        return None

def train_isolation_forest(features_list):
    # Convert feature list into pandas DataFrame
    df = pd.DataFrame(features_list)

    # Ensure that all columns are present (even if some values are missing)
    all_columns = ['length', 'src_mac', 'dst_mac', 'arp_opcode', 'arp_src_ip', 'arp_dst_ip',
                   'src_ip', 'dst_ip', 'ip_len', 'ip_ttl', 'src_tcp_port', 'dst_tcp_port',
                   'tcp_flags', 'seq_number', 'ack_number', 'tcp_window_size', 'tls_record_type',
                   'tls_version', 'src_udp_port', 'dst_udp_port']

    for col in all_columns:
        if col not in df.columns:
            df[col] = None

    # Handle missing values (replace with mean or median)
    df = df.fillna(df.mean())

    # Normalize the features
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)

    # Train Isolation Forest model
    model = IsolationForest(contamination=0.05)  # Assuming 5% of the data may be anomalous
    model.fit(df_scaled)

    return model, scaler

def capture_packets_and_detect_anomalies(model, scaler):
    cap = pyshark.LiveCapture(interface='en0')  # Adjust the interface

    # Define the column structure here to match the training data
    all_columns = ['length', 'src_mac', 'dst_mac', 'arp_opcode', 'arp_src_ip', 'arp_dst_ip',
                   'src_ip', 'dst_ip', 'ip_len', 'ip_ttl', 'src_tcp_port', 'dst_tcp_port',
                   'tcp_flags', 'seq_number', 'ack_number', 'tcp_window_size', 'tls_record_type',
                   'tls_version', 'src_udp_port', 'dst_udp_port']

    # Capture packets continuously and check for anomalies
    for packet in cap.sniff_continuously(packet_count=10):
        features = extract_features(packet)
        if features:
            # Convert features into a DataFrame row (same format as the model training data)
            df = pd.DataFrame([features])

            # Ensure all necessary columns are included (fill missing columns)
            for col in all_columns:
                if col not in df.columns:
                    df[col] = None

            # Normalize the new packet features
            df_scaled = scaler.transform(df)

            # Predict anomaly using Isolation Forest
            prediction = model.predict(df_scaled)

            # If prediction is -1, it's an anomaly
            if prediction == -1:
                print(f"Anomaly detected in packet: {packet}")
            else:
                print(f"Normal packet: {packet}")


In [ ]:
import pyshark
import pandas as pd

def collect_historical_data(interface, capture_time=60, packet_count=1000):
    # Use pyshark to capture packets
    cap = pyshark.LiveCapture(interface=interface)

    # List to store packet features
    packet_features = []

    # Collect packets
    print(f"Capturing {packet_count} packets for training...")
    for packet in cap.sniff_continuously(packet_count=packet_count):
        features = extract_features(packet)
        if features:
            packet_features.append(features)

    # Convert the list of features to a DataFrame
    df = pd.DataFrame(packet_features)

    # Save the historical data to a CSV file for future use
    df.to_csv('historical_packet_data.csv', index=False)
    print("Historical data saved to 'historical_packet_data.csv'")
    return df


In [ ]:
# Collect historical data
historical_data = collect_historical_data(interface='en0', capture_time=60, packet_count=1000)


Capturing 1000 packets for training...
Historical data saved to 'historical_packet_data.csv'


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

def train_isolation_forest(features_list):
    # Assuming features_list is a DataFrame
    df = pd.DataFrame(features_list)

    # Impute missing values with the mean of each column
    imputer = SimpleImputer(strategy='mean')
    df_imputed = imputer.fit_transform(df)

    # Normalize the data
    scaler = StandardScaler()
    df_normalized = scaler.fit_transform(df_imputed)

    # Train the Isolation Forest model
    model = IsolationForest(contamination=0.05, random_state=42)
    model.fit(df_normalized)

    return model, scaler

# Apply the function to the historical data
model, scaler = train_isolation_forest(historical_data)
print("Model trained successfully.")


Model trained successfully.


/opt/anaconda3/lib/python3.12/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['arp_opcode' 'arp_src_ip' 'arp_dst_ip' 'tcp_flags' 'tls_record_type'
 'tls_version' 'src_udp_port' 'dst_udp_port']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [ ]:
def train_isolation_forest(features_list):
    df = pd.DataFrame(features_list)

    # Drop columns with all missing values
    df = df.dropna(axis=1, how='all')

    # Impute missing values with the mean of each column
    imputer = SimpleImputer(strategy='mean')
    df_imputed = imputer.fit_transform(df)

    # Normalize the data
    scaler = StandardScaler()
    df_normalized = scaler.fit_transform(df_imputed)

    # Train the Isolation Forest model
    model = IsolationForest(contamination=0.05, random_state=42)
    model.fit(df_normalized)

    return model, scaler

# Apply the function to the historical data
model, scaler = train_isolation_forest(historical_data)
print("Model trained successfully.")


Model trained successfully.


In [ ]:
# Function to detect anomalies in new data
def detect_anomalies(model, scaler, new_data):
    # Normalize the new data using the same scaler as the training data
    new_data_normalized = scaler.transform(new_data)

    # Predict anomalies: -1 for outliers, 1 for normal points
    predictions = model.predict(new_data_normalized)

    return predictions

# Example of how to use the model to predict anomalies on new data:
new_data = pd.DataFrame(new_data_features)  # Assuming new_data_features is your new data
predictions = detect_anomalies(model, scaler, new_data)

# Show the results
print("Anomaly predictions:", predictions)


NameError: name 'new_data_features' is not defined